In [13]:
import keras_tuner as kt
import tensorflow_datasets as tfds
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

a = tfds.load('german_credit_numeric')
df = tfds.as_dataframe(a['train'])
features = pd.DataFrame(df['features'].tolist()).values
targets = df['label'].values


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

X_train, X_test, y_train,y_test = train_test_split(features, targets)

scaler = StandardScaler().fit(X_train)
X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,...,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02
mean,-2.232288e-16,-1.243450e-17,6.098825e-17,-3.597123e-17,-1.181277e-16,1.894781e-17,-5.921189e-18,-1.551352e-16,3.780679e-16,-2.285579e-16,...,1.042869e-16,5.234331e-16,2.275217e-16,-8.230453e-16,-3.390621e-16,-1.697901e-16,-1.496581e-16,5.092223e-17,-5.921189e-18,2.593481e-16
std,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,...,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00
min,-1.255751e+00,-1.421168e+00,-2.327800e+00,-1.090848e+00,-6.914318e-01,-1.988075e+00,-2.331623e+00,-1.705412e+00,-1.296348e+00,-1.447498e+00,...,-2.111119e-01,-5.598974e-01,-3.357970e-01,-3.068659e+00,-2.111119e-01,-4.579055e-01,-1.598281e+00,-1.522904e-01,-4.916522e-01,-1.337200e+00
25%,-1.255751e+00,-7.470431e-01,-4.744560e-01,-6.914204e-01,-6.914318e-01,-3.230206e-01,-9.247708e-01,-7.950499e-01,-1.296348e+00,-7.574464e-01,...,-2.111119e-01,-5.598974e-01,-3.357970e-01,3.258753e-01,-2.111119e-01,-4.579055e-01,-1.598281e+00,-1.522904e-01,-4.916522e-01,-1.337200e+00
50%,-4.582802e-01,-2.414492e-01,-4.744560e-01,-3.283049e-01,-6.914318e-01,-3.230206e-01,4.820813e-01,1.153126e-01,-3.440846e-01,-2.399079e-01,...,-2.111119e-01,-5.598974e-01,-3.357970e-01,3.258753e-01,-2.111119e-01,-4.579055e-01,6.256722e-01,-1.522904e-01,-4.916522e-01,7.478312e-01
75%,1.136662e+00,2.641447e-01,1.378888e+00,2.526800e-01,5.858437e-01,1.342034e+00,4.820813e-01,1.025675e+00,6.081791e-01,5.363999e-01,...,-2.111119e-01,-5.598974e-01,-3.357970e-01,3.258753e-01,-2.111119e-01,-4.579055e-01,6.256722e-01,-1.522904e-01,-4.916522e-01,7.478312e-01
max,1.136662e+00,3.297708e+00,1.378888e+00,5.517855e+00,1.863119e+00,1.342034e+00,1.888934e+00,1.025675e+00,1.560443e+00,3.382862e+00,...,4.736824e+00,1.786041e+00,2.977989e+00,3.258753e-01,4.736824e+00,2.183857e+00,6.256722e-01,6.566403e+00,2.033958e+00,7.478312e-01


In [14]:
from CustomLayers import FeatureReversalNoise
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(24)))
model.add(FeatureReversalNoise(input_vector_length=24, prob=.2))
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(1, activation='tanh'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.01), loss='mse')
model.fit(x=X_train,y=y_train, verbose=0, epochs=10, validation_split=.2)
y_preds = model.predict(X_test)
mean_squared_error(y_test,y_preds)

0.1644783814502942

In [15]:
from tensorflow import keras
def model_builder_factory(n_features,
                          prob,
                          min_layers,
                          max_layers,
                          min_hidden_nodes,
                          max_hidden_nodes,
                          include_prob):
  """
  returns the model_builder function based on the hyper parameters passed 
  """
  def hypermodel(hp):
    glorrot_init = tf.keras.initializers.GlorotUniform() # A smarter way to set inital weights
    model = keras.Sequential()
    model.add(tf.keras.Input(shape=(n_features)))
    if include_prob:
      model.add(FeatureReversalNoise(input_vector_length=n_features,prob=prob))
    
    nodes_per_layer = []
    for i in range(max_layers):
      hp_units = hp.Int(f'Dense_layer{i}',
                      min_value=min_hidden_nodes,
                      max_value=max_hidden_nodes,
                      step=1)
      nodes_per_layer.append(hp_units)

    for i in range(hp.Int('n_layers', min_value=min_layers, max_value=max_layers, step=1)):
      model.add(keras.layers.Dense(units=nodes_per_layer[i],
                                   activation='relu',
                                   kernel_initializer=glorrot_init))
      
    model.add(keras.layers.Dense(1,activation='tanh'))
    # hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3,])
    hp_beta_1 = hp.Choice('beta_1', values=[.8,.9,.92,.94,.95])
    hp_beta_2 = hp.Choice('beta_2', values=[.98,.99,.995,.999])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=.01,
                                                  beta_1=hp_beta_1,
                                                  beta_2=hp_beta_2),
                  loss='mse', metrics=['mse'])

    return model

  return hypermodel


hypermodel = model_builder_factory(24,.1,1,3,2,8,include_prob=True)
tuner = kt.RandomSearch(hypermodel=hypermodel,
                          objective='mse',
                          max_trials=10,
                          overwrite=True,
                          #directory='/kerasproject',
                          project_name='testLayers')
#stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=3)
tuner.search(x=X_train,y=y_train,      
            verbose=0,
            epochs=10,
            batch_size=2**8)

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [17]:
best_hps = tuner.get_best_hyperparameters(num_trials=5)[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.fit(X_train,y_train,epochs=20,batch_size=2**8,verbose=0)
y_preds_in_memory = best_model.predict(X_test)

best_model.save(f'model.h5',save_format='h5')
saved_model = tf.keras.models.load_model("model.h5")
y_preds_from_disk = saved_model.predict(X_test)
print('Are the predictions of the model in memory are the same as the model loaded from disk?')
np.all(y_preds_in_memory == y_preds_from_disk)

The predictions of the model in memory are the same as the model loaded from disk


True

These values should be the same. If they are not that means you got an error